# Apparent Loss Project

In [ ]:
import warnings
import time
from glob import glob

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

from dash import Input, Output, dcc, html
from jupyter_dash import JupyterDash

warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
def wrangle(file):
    # Import the file in excel format which is advisable to avoid losing some info
    df = pd.read_excel(file)
    
    # Obtain dates only and exclude time
    df["bill_date"] = df["bill_date"].astype(str).str.split(" ", expand=True)[0]
    
    # Handling the occurance of many months in one excel sheet
    nbr_of_months = sum(df["bill_date"].str[:7].value_counts(normalize=True) > .30)
    
    # for stacked months in a excel file
    if nbr_of_months == 3:
        dfs = []
        months = df["bill_date"].str[:7].value_counts().sort_values().tail(3).index.to_list()
        for month in months:
            frame = df[df["bill_date"].apply(lambda date: month in date)]
            frame = frame.drop_duplicates(subset=["customer_identifier"], keep="last")
            dfs.append(frame)
            
        return dfs
    
    # for single month in a excel file
    else:
        month = df["bill_date"].str[:7].value_counts().sort_values(ascending=True).tail(1).index[0]
        df = df[df["bill_date"].apply(lambda date: month in date)]
        df = df.drop_duplicates(subset=["customer_identifier"], keep="last")
        
        return df

In [ ]:
files = glob("excelfiles/20*.xlsx")

files

In [ ]:
!pip install openpyxl

In [ ]:
# Start time
start = time.time()

frames = [wrangle(file) for file in files]

# End time
end = time.time()

print("Elapsed time:", end - start)

In [ ]:
len(frames), len(frames[0]), len(frames[1])

In [ ]:
frames[0][0].head()

In [ ]:
list_of_df = []
list_of_df.extend(frames[0])
list_of_df.extend(frames[1])
list_of_df.extend(frames[2:])

In [ ]:
len(list_of_df)

In [ ]:
list_of_df[9].head()

In [ ]:
months = ["csv_files/" + str(i) + ".csv" for i in range(201907, 201913)]
months.extend(["csv_files/" + str(i) + ".csv" for i in range(202001, 202013)])
months.extend(["csv_files/" + str(i) + ".csv" for i in range(202101, 202107)])

months

In [ ]:
len(months)

In [ ]:
start = time.time()

i = 0
for frame in list_of_df:
    frame.to_csv(months[i], index=False)
    i+=1
    
end = time.time()

print("Elapsed time:", end - start)

In [ ]:
def wrangle(filepath):
    df = pd.read_csv(filepath, usecols=["bill_date", "customer_identifier", "consumption"])
    return df

In [ ]:
files = glob("csv_files/20*.csv")
files

In [ ]:
start = time.time()

frames = [wrangle(file) for file in files]
    
end = time.time()

print("Elapsed time:", end - start)

In [ ]:
frames[23].head()

In [ ]:
start = time.time()

water_cons_df = frames[0].drop(columns="bill_date")
for frame in frames[1:]:
    water_cons_df = pd.merge(
        water_cons_df, 
        frame.drop(columns="bill_date"), 
        on="customer_identifier", how="inner"
    )
    
end = time.time()
print("Elapsed time:", end - start)

In [ ]:
print(water_cons_df.shape)
water_cons_df.head()

In [ ]:
columns = ["customer_identifier"]
columns.extend(pd.date_range('2019-06-10','2021-06-07', freq='MS').strftime("%Y-%b").tolist())
    
water_cons_df.columns = columns

In [ ]:
print(water_cons_df.shape)
print(water_cons_df.columns)
water_cons_df.head()

In [ ]:
water_cons_df.info()

In [ ]:
water_cons_df.to_csv("water_cons_data.csv", index=False)

In [ ]:
df = pd.read_csv("water_cons_data.csv")

In [ ]:
print(df.shape)
df.head()

In [ ]:
df_sample = df.sample(frac=.05, random_state=42)

print(df_sample.shape)
df_sample.head()

In [ ]:
df_sample.set_index("customer_identifier", inplace=True)

In [ ]:
df_sample.head()

In [ ]:
model = make_pipeline(
    StandardScaler(), KMeans(n_clusters=2, random_state=42)
)

model.fit(df_sample)

In [ ]:
pca = PCA(n_components=2, random_state=42)
df_sample_tr = pd.DataFrame(pca.fit_transform(df_sample), columns=["PCA1", "PCA2"], index=df_sample.index)

In [ ]:
print(df_sample_tr.shape)
df_sample_tr.head()

In [ ]:
df_sample_tr["labels"] = model.named_steps["kmeans"].labels_.astype(str)

In [ ]:
df_sample_tr.head()

In [ ]:
df_sample_tr["labels"].value_counts()

In [ ]:
df_sample[df_sample_tr["labels"] == "1"]

In [ ]:
fig = px.scatter(
        data_frame=df_sample_tr,
        x="PCA1", y="PCA2", color="labels",
        title="PCA Representation of Clusters"
)
    
fig.update_layout(xaxis_title="PCA1", yaxis_title="PCA2")

In [ ]:
app = JupyterDash(__name__)

In [ ]:
app.layout = html.Div(
    [
        html.H1("Tracking Apparent Losses in Water Bills")
    ]
)

In [ ]:
app.layout = html.Div(
    [
        html.H1("Survey of Consumer Finances"),
        html.H2("High Variance Features"),
        dcc.Graph(id="bar-chart"),
        dcc.RadioItems(
            options=[
                {"label": "trimmed", "value": True},
                {"label": "not trimmed", "value": False}
            ],
            value=True,
            id="trim-button"
        ),
        html.H2("K-means Clustering"),
        html.H3("Number of Clusters (k)"),
        dcc.Slider(min=2, max=12, step=1, value=2, id="k-slider"),
        html.Div(id="metrics"),
        dcc.Graph(id="pca-scatter")
    ]
)

In [ ]:
def get_high_var_features(trimmed=True, return_feat_names=True):

    """Returns the five highest-variance features of ``df``.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    return_feat_names : bool, default=False
        If ``True``, returns feature names as a ``list``. If ``False``
        returns ``Series``, where index is feature names and values are
        variances.
    """
    if trimmed:
        top_five_features = df.apply(trimmed_var).sort_values().tail(5)
    else:
        top_five_features = df.var().sort_values().tail(5)
        
    if return_feat_names:
        top_five_features = top_five_features.index.to_list()
    
    return top_five_features

In [ ]:
@app.callback(
    Output("bar-chart", "figure"), Input("trim-button", "value")
)
def serve_bar_chart(trimmed=True):

    """Returns a horizontal bar chart of five highest-variance features.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.
    """
    top_five_features = get_high_var_features(trimmed=trimmed, return_feat_names=False)
    
    fig = px.bar(x=top_five_features, y=top_five_features.index, orientation="h")
    fig.update_layout(xaxis_title="Variance", yaxis_title="Feature")
    
    return fig

In [ ]:
def get_model_metrics(trimmed=True, k=2, return_metrics=False):

    """Build ``KMeans`` model based on five highest-variance features in ``df``.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.

    return_metrics : bool, default=False
        If ``False`` returns ``KMeans`` model. If ``True`` returns ``dict``
        with inertia and silhouette score.

    """
    features = get_high_var_features(trimmed=trimmed, return_feat_names=True)
    X = df[features]
    model = make_pipeline(StandardScaler(), KMeans(n_clusters=k, random_state=42))
    model.fit(X)
    
    if return_metrics:
        i = model.named_steps["kmeans"].inertia_
        ss = silhouette_score(X, model.named_steps["kmeans"].labels_)
        metrics = {
            "inertia": round(i),
            "silhouette": round(ss, 3)
        }
        
        return metrics
    
    return model

In [ ]:
@app.callback(
    Output("metrics", "children"),
    Input("trim-button", "value"),
    Input("k-slider", "value")
)
def serve_metrics(trimmed=True, k=2):

    """Returns list of ``H3`` elements containing inertia and silhouette score
    for ``KMeans`` model.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.
    """
    metrics = get_model_metrics(trimmed=trimmed, k=k, return_metrics=True)
    text = [
        html.H3(f"Inertia: {metrics['inertia']}"),
        html.H3(f"Silhouette Score: {metrics['silhouette']}")
    ]
    
    return text

In [ ]:
def get_pca_labels(trimmed=True, k=2):

    """
    ``KMeans`` labels.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.
    """
    features = get_high_var_features(trimmed=trimmed, return_feat_names=True)
    X = df[features]
    transformer = PCA(n_components=2, random_state=42)
    X_t = transformer.fit_transform(X)
    X_pca = pd.DataFrame(X_t, columns=["PCA1", "PCA2"])
    model = get_model_metrics(trimmed=trimmed, k=k, return_metrics=False)
    X_pca["labels"] = model.named_steps["kmeans"].labels_.astype(str)
    
    X_pca.sort_values("labels", inplace=True)
    
    return X_pca

In [ ]:
@app.callback(
    Output("pca-scatter", "figure"),
    Input("trim-button", "value"),
    Input("k-slider", "value")
)
def serve_scatter_plot(trimmed=True, k=2):

    """Build 2D scatter plot of ``df`` with ``KMeans`` labels.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    k : int, default=2
        Number of clusters.
    """
    fig = px.scatter(
        data_frame=get_pca_labels(trimmed=trimmed, k=k),
        x="PCA1", y="PCA2", color="labels",
        title="PCA Representation of Clusters"
    )
    
    fig.update_layout(xaxis_title="PCA1", yaxis_title="PCA2")
    
    return fig

In [ ]:
app.run_server(host="0.0.0.0", mode="external")